# Query DB for Emails

In [1]:
%load_ext autoreload
%autoreload 2

from MySQL_Scripts.modules.acquiring_db_emails import query_database, drop_unsubscribed


#Query the DB for the customer base that you are narrowing down to, and drop unsubscribed emails
mlb = [
    'Angels', 'Astros', 'Athletics', 'Blue Jays', 'Braves', 'Brewers', 'Cardinals', 'Cubs', 'Diamondbacks', 'Dodgers',
    'Giants', 'Indians', 'Mariners', 'Marlins', 'Mets', 'Nationals', 'Orioles', 'Padres', 'Phillies', 'Pirates',
    'Rangers', 'Rays', 'Red Sox', 'Reds', 'Rockies', 'Royals', 'Tigers', 'Twins', 'White Sox', 'Yankees', 'Baseball', 'MLB'
]

# Create the SQL query string
query = f'''
    SELECT * 
    FROM email_prospecting
    WHERE {' OR '.join([f"LOWER(product_name) LIKE '%{x.lower()}%'" for x in mlb])};
'''

# Assuming you have a function query_database defined
df = query_database(query)
df = drop_unsubscribed(df)

#This is then references from a csv in the sends

Tunnel local bind port: 63142
Tunnel is active: True
Connection is closed
Tunnel local bind port: 63242
Tunnel is active: True
Connection is closed


# Establish Process to Blast

In [2]:
from config import imap_password_customplanet, db_username, db_password
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import time
from Sending_Emails.modules.sends import SendMail

EMAIL_ADDRESS_FROM = 'team@customplanet.com'
EMAIL_PASS = imap_password_customplanet
# Specify database connection details
server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
database = 'emailcampaign'
db = 'emailcampaign'
table_name = 'email_history'

def process(df, contact_column, sport, email_campaign_name, global_subject_line, school=None):

    data_list = []
    processed_emails = set()

    #Limit df itterrows to test
    for index, row in df[:2].iterrows():

 
        #Set the time zone to Central Time
        central_time_zone = pytz.timezone('America/Chicago')
        now_central = datetime.now(central_time_zone)
        formatted_date = now_central.strftime("%m/%d/%Y")
        #When running campaigns to non schools
        try:
            school = row['School Name']
        except:
            school = None

        # png_path = row['PNG_PATH']
        contact = row[contact_column]
  
        # Check if the email has already been processed
        if contact in processed_emails:
            print(f"Skipping email to {contact} as it has already been processed.")
            continue

        data = [school, contact, sport, formatted_date]
        data_list.append(data)

        # Mark the email as processed
        processed_emails.add(contact)
        
        SendMail.send(EMAIL_ADDRESS_FROM, EMAIL_PASS, global_subject_line, school, sport, contact)

    
    data_list = pd.DataFrame(data_list, columns=['School', 'Contact', 'Sport', 'Date_Sent'])

    data_list.rename(columns = {'Contact': 'contact_email', 'School': 'school', 'Date_Sent': 'date_sent', 'Sport': 'sport'}, inplace = True)
    data_list['date'] = data_list['date_sent'].astype(str)

    data_list['subject'] = global_subject_line
    data_list['position'] = contact_column
    data_list['from'] = EMAIL_ADDRESS_FROM
    data_list['email_campaign_tag'] = email_campaign_name
    data_list['date'] = now_central.strftime("%Y-%m-%d %H:%M:%S")

    #This is present in case the process breaks it knows where to resume
    data_list.to_csv('output.csv', index=False)

    return(data_list)

# --------------------------------------Running Process--------------------------

#Temporary testing to modify the Athletic Director email to my own
df.loc[0, 'email'] = '2015samtaylor@gmail.com'
df.loc[1, 'email'] = 'sammytaylor2006@yahoo.com'
df.loc[2, 'email'] = 'jerrybons2006@gmail.com'
# df.loc[2, 'email'] = 'samuel.taylor@greendot.org'
# df.loc[3, 'email'] = 'sammtaylor2006@gmail.com'


#Global Subject line references format string in sends.py, and is also the subject line of the email
#Contact_Column is the column coming from df that passes in the email


In [3]:
global global_subject_line
global_subject_line = 'Official MLB Jerseys'
school = None

email_history = process(df, 'email' ,'Baseball', 'Official MLB Jerseys in Stock!!', global_subject_line)

#Test this out to get more understanding on the Blast function.
#Should the template be an argument

In [4]:
# pd.read_csv('output.csv')

email_history

#This is 

,school,contact_email,sport,date_sent,date,subject,position,from,email_campaign_tag
0,None,2015samtaylor@gmail.com,Baseball,02/20/2024,2024-02-20 16:10:47,Official MLB Jerseys,email,team@customplanet.com,Official MLB Jerseys in Stock!!
1,None,sammytaylor2006@yahoo.com,Baseball,02/20/2024,2024-02-20 16:10:47,Official MLB Jerseys,email,team@customplanet.com,Official MLB Jerseys in Stock!!


# Calling the Function to Send Emails Iteratively

In [ ]:
#The logic of this function works by declaring the email_history variable which is returned from the process function
# This is a dataframe of the emails who it has been sent to. Also is written to output.csv


def blast(df):

    # Get the next 50 emails iteratively and send every 60 seconds with a new SMTP connection
    # get_next_50 is what ends this function. It will break one end_point has surpassed.
    while len(df) >= 1:

        df = SendMail.get_next_50(df, email_history)

        new_point = df.index[0]
        end_point = df.index[-1]   
        print(new_point, end_point) 
        
        # Process the emails
        email_history = process(df, 'email' ,'Baseball', 'Official MLB Jerseys in Stock!!', global_subject_line)
        # email_history = process(df, contact_column, sport, email_campaign_name, global_subject_line=global_subject_line, school=None)
      
        interval_seconds = 60
        # Wait for the specified interval before the next iteration
        time.sleep(interval_seconds)


global global_subject_line
global_subject_line = 'Official MLB Jerseys'
school = None
blast(df)


#Sends 500 emails in 15 mins
#Limit to 1500 emails per day
#Anything over 2000 the entire account gets locked for 24 hours. 
#Limits to 2000 emails per day
# SMTPDataError: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 w4-20020a4ae9e4000000b005914f455774sm848190ooc.34 - gsmtp')

In [ ]:
#THis needs to be logged out properly. Do this with the testing

https://imgur.com/a/F9azawI

# https://lh3.googleusercontent.com/drive-viewer/AEYmBYSC8neSpHDVZ85b0HNeRJGBPIzhCrSENTUy4I552Rhz1OrdKTQXSFBFhb0JlBgQxww1Yd-ydVUkKmAygDhhEfi-cT0NIQ=s1600